# Description

In [ ]:
#
#    Numerical sumulations
#    One Dimensional Modeling of Plant Growth

#    This program models plant growth in one dimension. 
#    numerical method to solve the ordinary differential equations describing
#    plant growth
#        - Euler Method
#        
    
#    It also solves the one-dimensional diffusion-advection equation, using the 
#          - Crank Nicolson Method.
    
#    Equations to solve:
    #    - dL/dx = f(R,L)    where x = L(t), x is the end of the plant. where the expression of f are:
         # if R > Rf
                 # linear: f(R,L) = f0
                 # Logisctic: f(R,L) = r' *L*( 1 - ( (L**q) / (k'**q) ) )
                 # Gompertz: f(R,L) = c*L*( np.log(k'/L))
                 # Aikman & Scaife: f(R,L) = (r_*L*( (1 - np.exp(-n*KF*p*(L**q) )))) / (n*KF*p*(L**q)) 
        # else f = 0
        # x= EDD & z for space
    #    - h*(dR/dx) = g(R)*C - sigma*R
    #    - dC/dx = -u*dC/dz + d*d^2C/dz^2, the diffusion-advection equation
    


# linear

In [5]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.sparse import diags
from matplotlib.animation import writers
%matplotlib
plt.ion()

def f(R):
    if 0 <= R <= Rf:
        f = 0
    elif R > Rf:
        f = f0
    else:
        print('Error: R must be greater than 0')
    return f


def g(R):
    if 0 <= R <= Rg1:
        g = (g0/Rg1) * R
    elif Rg1 < R < Rg2:
        g = g0
    elif R >= Rg2:
        g = 0
    else:
        print('Error: R must be greater than 0')
    return g

g0=0.01
ht = 0.0001
tf = 3000
N = 101

sigma = 0.009  # Growth factor decay coefficient
w = 0.05  # Width of meristem
d = 0.001  # Diffusion coefficient (of metabolites in xylem fluid)
Rf = 0.04 # Critical values of R and G functions
Rg1 = 0.01  # " "
Rg2 = 1  # " "
f0 = 0.0003075  # " "

# Initial Conditions
Rplot = [1.]# Initial Conditions

# Rplot = [Rf / 2.0] # Growth factor starts below critical value
Lplot = [0.1]  # Plant length is 0 at t = 0
Cplot = [1.]
tplot = [0]

# Initiate a grid (for concentration of metabolites)
h = Lplot[0] / (N - 1)  # Spacing of grid
x = np.arange(0, N) * h  # Initial coordinates of grid points
C = np.ones(N)  # Initiate C (concentration of metabolites)
C[0] = 1  # Boundary Condition

# Construct State Vector
state = np.array([Rplot[0], Lplot[0]])


# %% Numerical Method Functions


def rk4(x, C, t, ht, f):
    F1 = f(x, C, t)
    F2 = f(x + (0.5 * ht) * F1, C, t + 0.5 * ht)
    F3 = f(x + (0.5 * ht) * F2, C, t + 0.5 * ht)
    F4 = f(x + ht * F3, C, t + ht)
    xout = x + ht / 6. * (F1 + F4 + 2. * (F2 + F3))
    return xout

b = (ht * d) / (2 * (h ** 2))  # Final parameter (depends on h so must be defined here)

def crank(s, C_in):
    N = len(C_in)
    a = (ht * f(s[0])) / (4 * h)
    M1 = diags([-a - b, 1 + 2 * b, a - b], [-1, 0, 1], shape=(N, N)).toarray()
    M2 = diags([b + a, 1 - 2 * b, b - a], [-1, 0, 1], shape=(N, N)).toarray()
    M = np.linalg.inv(M1).dot(M2)
    C_out = C_in.dot(M)  # written this way because C is a row vector
    return C_out


def derivated(s, C, t):
    aux = s + ((C[-1]) * g(s) * ht - sigma * s * ht) / (0.0001*w)
    return aux


time = 0.
for istep in range(0, tf):  ###  MAIN LOOP  ###
    L0 = np.copy(state[1])
    state[0] = derivated(state[0], C, istep)
    time = (time + ht)
    
    C = crank(state, C)
    dL = state[1] - L0  # Change in L
    dN = int(np.round(dL / h))  # Number of new grid points

    # Apply BCs to fill in new grid points.
    C[0] = 1
    
    Ci = (1 - (g(state[0]) * (h / d))) * C[-1]  # d dC/dx = - g(R)C
    if dL > 0:
        C = np.append(C, dN * [Ci])
    else:
        C[-1] = Ci
    if (istep % 1000) == 0:
        print('Number of iterations completed:', istep)
    Rplot.append(state[0])
    state[1] = Lplot[-1] + f(state[0])
    Lplot.append(state[1])
    Cplot.append(C[-1])
    tplot.append(istep+1)
    
# Non-animated plots of L(t) and R(t)
plt.subplot(131)
plt.plot(tplot, Lplot, label='Linear', color='#1f77b4')
plt.title('Length of Plant L vs. EDD');
plt.xlabel('EDD');
plt.ylabel('Length');
plt.legend(loc='lower right')
plt.grid(True, which='both', axis='both', linestyle='--')
z=max(tplot)
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0.1, 0.74+0.064, 0.064))

plt.subplot(132)
plt.plot(tplot, Rplot, color='#1f77b4')
plt.title('Growth Factor Concentration R vs. EDD');
plt.xlabel('EDD');
plt.ylabel('R')
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))




plt.subplot(133)
plt.plot(tplot, Cplot, color='#1f77b4')
plt.title('Metabolite Concentration C at end point vs. EDD');
plt.xlabel('EDD');
plt.ylabel('C')
plt.grid()
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))

R_1=Rplot
C_1=Cplot









Using matplotlib backend: TkAgg
Number of iterations completed: 0
Number of iterations completed: 1000
Number of iterations completed: 2000


# Logistic

In [6]:


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.sparse import diags
from matplotlib.animation import writers
%matplotlib
plt.ion()
p=0.41
q=3.3
k=0.13489
r=0.01
k_=(k/p)**(1/q)
r_=r/q
def f(R,L):
    if 0 <= R <= Rf:
        f = 0
    elif R > Rf:
        f = r_*L*( 1 - ( (L**q) / (k_**q) ) )
    else:
        print('Error: R must be greater than 0')
    return f


def g(R):
    if 0 <= R <= Rg1:
        g = (g0/Rg1) * R
    elif Rg1 < R < Rg2:
        g = g0
    elif R >= Rg2:
        g = 0
    else:
        print('Error: R must be greater than 0')
    return g

g0=0.01
ht = 0.0001
tf = 3000
N = 101

sigma = 0.009  # Growth factor decay coefficient
w = 0.05  # Width of meristem
d = 0.001  # Diffusion coefficient (of metabolites in xylem fluid)
Rf = 0.04 # Critical values of R and G functions
Rg1 = 0.01  # " "
Rg2 = 1  # " "
f0 = 0.0003075  # " "

# Initial Conditions
Rplot = [1.]# Initial Conditions

# Rplot = [Rf / 2.0] # Growth factor starts below critical value
Lplot = [0.1]  # Plant length is 0 at t = 0
Cplot = [1.]
tplot = [0]

# Initiate a grid (for concentration of metabolites)
h = Lplot[0] / (N - 1)  # Spacing of grid
x = np.arange(0, N) * h  # Initial coordinates of grid points
C = np.ones(N)  # Initiate C (concentration of metabolites)
C[0] = 1  # Boundary Condition

# Construct State Vector
state = np.array([Rplot[0], Lplot[0]])


# %% Numerical Method Functions


def rk4(x, C, t, ht, f):
    F1 = f(x, C, t)
    F2 = f(x + (0.5 * ht) * F1, C, t + 0.5 * ht)
    F3 = f(x + (0.5 * ht) * F2, C, t + 0.5 * ht)
    F4 = f(x + ht * F3, C, t + ht)
    xout = x + ht / 6. * (F1 + F4 + 2. * (F2 + F3))
    return xout

b = (ht * d) / (2 * (h ** 2))  # Final parameter (depends on h so must be defined here)

def crank(s, C_in):
    N = len(C_in)
    a = (ht * f(s[0], s[1])) / (4 * h)
    M1 = diags([-a - b, 1 + 2 * b, a - b], [-1, 0, 1], shape=(N, N)).toarray()
    M2 = diags([b + a, 1 - 2 * b, b - a], [-1, 0, 1], shape=(N, N)).toarray()
    M = np.linalg.inv(M1).dot(M2)
    C_out = C_in.dot(M)  # written this way because C is a row vector
    return C_out


def derivated(s, C, t):
    aux = s + ((C[-1]) * g(s) * ht - sigma * s * ht) / (0.0001*w)
    return aux


time = 0.
for istep in range(0, tf):  ###  MAIN LOOP  ###
    L0 = np.copy(state[1])
    state[0] = derivated(state[0], C, istep)
    time = (time + ht)
    
    C = crank(state, C)
    dL = state[1] - L0  # Change in L
    dN = int(np.round(dL / h))  # Number of new grid points

    # Apply BCs to fill in new grid points.
    C[0] = 1
    
    Ci = (1 - (g(state[0]) * (h / d))) * C[-1]  # d dC/dx = - g(R)C
    if dL > 0:
        C = np.append(C, dN * [Ci])
    else:
        C[-1] = Ci
    if (istep % 1000) == 0:
        print('Number of iterations completed:', istep)
    Rplot.append(state[0])
    #state[1] = Lplot[-1] * (1 + (r_*(1-((Lplot[-1]**q)/(k_**q))) ))
    state[1] = Lplot[-1] + f(state[0], Lplot[-1])
    Lplot.append(state[1])
    Cplot.append(C[-1])
    tplot.append(istep+1)
    
# Non-animated plots of L(t) and R(t)

plt.subplot(131)
plt.plot(tplot, Lplot, label='Logistic', color='#ff7f0e')
plt.title('Length of Plant L vs. EDD');
plt.xlabel('EDD');
plt.ylabel('Length');
plt.legend(loc='lower right')
plt.grid()
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0.1, 0.74+0.064, 0.064))

plt.subplot(132)
plt.plot(tplot, Rplot, color='#ff7f0e')
plt.title('Growth Factor Concentration R vs. EDD');
plt.xlabel('EDD');
plt.ylabel('R')
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0.1, 1+0.1, 0.1))



plt.subplot(133)
plt.plot(tplot, Cplot, color='#ff7f0e')
plt.title('Metabolite Concentration C at end point vs. EDD');
plt.xlabel('EDD');
plt.ylabel('C')
plt.grid()
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))


R_2=Rplot
C_2=Cplot


Using matplotlib backend: TkAgg
Number of iterations completed: 0
Number of iterations completed: 1000
Number of iterations completed: 2000


# Gompertz

In [7]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.sparse import diags
from matplotlib.animation import writers
%matplotlib
plt.ion()
p=0.41
q=3.3
k=0.13489
k_=(k/p)**(1/q)
c=0.0055
def f(R,L):
    if 0 <= R <= Rf:
        aaa = 0
    elif R > Rf:
        aaa = c*L*( np.log(k_/L))
    else:
        print('Error: R must be greater than 0')
    return aaa


def g(R):
    if 0 <= R <= Rg1:
        g = (g0/Rg1) * R
    elif Rg1 < R < Rg2:
        g = g0
    elif R >= Rg2:
        g = 0
    else:
        print('Error: R must be greater than 0')
    return g

g0=0.01
ht = 0.0001
tf = 3000
N = 101

sigma = 0.009  # Growth factor decay coefficient
w = 0.05  # Width of meristem
d = 0.001  # Diffusion coefficient (of metabolites in xylem fluid)
Rf = 0.04 # Critical values of R and G functions
Rg1 = 0.01  # " "
Rg2 = 1  # " "
f0 = 0.0031  # " "

# Initial Conditions
Rplot = [1.]# Initial Conditions

# Rplot = [Rf / 2.0] # Growth factor starts below critical value
Lplot = [0.1]  # Plant length is 0 at t = 0
Cplot = [1.]
tplot = [0]

# Initiate a grid (for concentration of metabolites)
h = Lplot[0] / (N - 1)  # Spacing of grid
x = np.arange(0, N) * h  # Initial coordinates of grid points
C = np.ones(N)  # Initiate C (concentration of metabolites)
C[0] = 1  # Boundary Condition

# Construct State Vector
state = np.array([Rplot[0], Lplot[0]])


# %% Numerical Method Functions


def rk4(x, C, t, ht, f):
    F1 = f(x, C, t)
    F2 = f(x + (0.5 * ht) * F1, C, t + 0.5 * ht)
    F3 = f(x + (0.5 * ht) * F2, C, t + 0.5 * ht)
    F4 = f(x + ht * F3, C, t + ht)
    xout = x + ht / 6. * (F1 + F4 + 2. * (F2 + F3))
    return xout

b = (ht * d) / (2 * (h ** 2))  # Final parameter (depends on h so must be defined here)

def crank(s, C_in):
    N = len(C_in)
    a = (ht * f(s[0], s[1])) / (4 * h)
    M1 = diags([-a - b, 1 + 2 * b, a - b], [-1, 0, 1], shape=(N, N)).toarray()
    M2 = diags([b + a, 1 - 2 * b, b - a], [-1, 0, 1], shape=(N, N)).toarray()
    M = np.linalg.inv(M1).dot(M2)
    C_out = C_in.dot(M)  # written this way because C is a row vector
    return C_out


def derivated(s, C, t):
    aux = s + ((C[-1]) * g(s) * ht - sigma * s * ht) / (0.0001*w)
    return aux


time = 0.
for istep in range(0, tf):  ###  MAIN LOOP  ###
    L0 = np.copy(state[1])
    state[0] = derivated(state[0], C, istep)
    time = (time + ht)
    
    C = crank(state, C)
    dL = state[1] - L0  # Change in L
    dN = int(np.round(dL / h))  # Number of new grid points

    # Apply BCs to fill in new grid points.
    C[0] = 1
    
    Ci = (1 - (g(state[0]) * (h / d))) * C[-1]  # d dC/dx = - g(R)C
    if dL > 0:
        C = np.append(C, dN * [Ci])
    else:
        C[-1] = Ci
    if (istep % 1000) == 0:
        print('Number of iterations completed:', istep)
    Rplot.append(state[0])
    #state[1] = Lplot[-1] * (1 + (r_*(1-((Lplot[-1]**q)/(k_**q))) ))
    state[1] = Lplot[-1] + f(state[0], Lplot[-1])
    Lplot.append(state[1])
    Cplot.append(C[-1])
    tplot.append(istep+1)
    
# Non-animated plots of L(t) and R(t)
plt.subplot(131)
plt.plot(tplot, Lplot, label='Gompertz', color='#2ca02c')
plt.title('Length of Plant L vs. EDD');
plt.xlabel('EDD');
plt.ylabel('Length');
plt.legend(loc='lower right')
plt.grid()
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0.1, 0.74+0.064, 0.064))

plt.subplot(132)
plt.plot(tplot, Rplot, color='#2ca02c')
plt.title('Growth Factor Concentration R vs. EDD');
plt.xlabel('EDD');
plt.ylabel('R')
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))






plt.subplot(133)
plt.plot(tplot, Cplot, color='#2ca02c')
plt.title('Metabolite Concentration C at end point vs. EDD');
plt.xlabel('EDD');
plt.ylabel('C')
plt.grid()
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))



R_3=Rplot
C_3=Cplot




Using matplotlib backend: TkAgg
Number of iterations completed: 0
Number of iterations completed: 1000
Number of iterations completed: 2000


# Aikman and Scaife

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.sparse import diags
from matplotlib.animation import writers
%matplotlib
plt.ion()
n=800
p=0.41
q=3.3
KF=0.59
r=0.033
r_=r/q
def f(R,L):
    if 0 <= R <= Rf:
        f = 0
    elif R > Rf:
        f = (r_*L*( (1 - np.exp(-n*KF*p*(L**q) )))) / (n*KF*p*(L**q)) 
    else:
        print('Error: R must be greater than 0')
    return f


def g(R):
    if 0 <= R <= Rg1:
        g = (g0/Rg1) * R
    elif Rg1 < R < Rg2:
        g = g0
    elif R >= Rg2:
        g = 0
    else:
        print('Error: R must be greater than 0')
    return g

g0=0.01
ht = 0.0001
tf = 3000
N = 101

sigma = 0.009  # Growth factor decay coefficient
w = 0.05  # Width of meristem
d = 0.001  # Diffusion coefficient (of metabolites in xylem fluid)
Rf = 0.04 # Critical values of R and G functions
Rg1 = 0.01  # " "
Rg2 = 1  # " "

# Initial Conditions
Rplot = [1.]# Initial Conditions

# Rplot = [Rf / 2.0] # Growth factor starts below critical value
Lplot = [0.1]  # Plant length is 0 at t = 0
Cplot = [1.]
tplot = [0]

# Initiate a grid (for concentration of metabolites)
h = Lplot[0] / (N - 1)  # Spacing of grid
x = np.arange(0, N) * h  # Initial coordinates of grid points
C = np.ones(N)  # Initiate C (concentration of metabolites)
C[0] = 1  # Boundary Condition

# Construct State Vector
state = np.array([Rplot[0], Lplot[0]])


# %% Numerical Method Functions


def rk4(x, C, t, ht, f):
    F1 = f(x, C, t)
    F2 = f(x + (0.5 * ht) * F1, C, t + 0.5 * ht)
    F3 = f(x + (0.5 * ht) * F2, C, t + 0.5 * ht)
    F4 = f(x + ht * F3, C, t + ht)
    xout = x + ht / 6. * (F1 + F4 + 2. * (F2 + F3))
    return xout

b = (ht * d) / (2 * (h ** 2))  # Final parameter (depends on h so must be defined here)

def crank(s, C_in):
    N = len(C_in)
    a = (ht * f(s[0], s[1])) / (4 * h)
    M1 = diags([-a - b, 1 + 2 * b, a - b], [-1, 0, 1], shape=(N, N)).toarray()
    M2 = diags([b + a, 1 - 2 * b, b - a], [-1, 0, 1], shape=(N, N)).toarray()
    M = np.linalg.inv(M1).dot(M2)
    C_out = C_in.dot(M)  # written this way because C is a row vector
    return C_out


def derivated(s, C, t):
    aux = s + ((C[-1]) * g(s) * ht - sigma * s * ht) / (0.0001*w)
    return aux


time = 0.
for istep in range(0, tf):  ###  MAIN LOOP  ###
    L0 = np.copy(state[1])
    state[0] = derivated(state[0], C, istep)
    time = (time + ht)
    
    C = crank(state, C)
    dL = state[1] - L0  # Change in L
    dN = int(np.round(dL / h))  # Number of new grid points

    # Apply BCs to fill in new grid points.
    C[0] = 1
    
    Ci = (1 - (g(state[0]) * (h / d))) * C[-1]  # d dC/dx = - g(R)C
    if dL > 0:
        C = np.append(C, dN * [Ci])
    else:
        C[-1] = Ci
    if (istep % 1000) == 0:
        print('Number of iterations completed:', istep)
    Rplot.append(state[0])
    #state[1] = Lplot[-1] * (1 + (r_*(1-((Lplot[-1]**q)/(k_**q))) ))
    state[1] = Lplot[-1] + f(state[0], Lplot[-1])
    Lplot.append(state[1])
    Cplot.append(C[-1])
    tplot.append(istep+1)
    
# Non-animated plots of L(t) and R(t)
plt.subplot(131)
plt.plot(tplot, Lplot, label='Scaife & Jones', color='#d62728')
plt.title('Length of Plant L vs. EDD');
plt.xlabel('EDD');
plt.ylabel('Length');
plt.legend(loc='lower right')
plt.grid()
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0.1, 0.74+0.064, 0.064))

plt.subplot(132)
plt.plot(tplot, Rplot, color='#d62728')
plt.title('Growth Factor Concentration R vs. EDD');
plt.xlabel('EDD');
plt.ylabel('R')
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))






plt.subplot(133)
plt.plot(tplot, Cplot, color='#d62728')
plt.title('Metabolite Concentration C at end point vs. EDD');
plt.xlabel('EDD');
plt.ylabel('C')
plt.grid()
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))




R_4=Rplot
C_4=Cplot
plt.show()


Using matplotlib backend: TkAgg
Number of iterations completed: 0
Number of iterations completed: 1000
Number of iterations completed: 2000


# Conclusion

In [1]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.sparse import diags
from matplotlib.animation import writers
%matplotlib
plt.ion()

def f(R):
    if 0 <= R <= Rf:
        f = 0
    elif R > Rf:
        f = f0
    else:
        print('Error: R must be greater than 0')
    return f


def g(R):
    if 0 <= R <= Rg1:
        g = (g0/Rg1) * R
    elif Rg1 < R < Rg2:
        g = g0
    elif R >= Rg2:
        g = 0
    else:
        print('Error: R must be greater than 0')
    return g

g0=0.01
ht = 0.0001
tf = 3000
N = 101

sigma = 0.009  # Growth factor decay coefficient
w = 0.05  # Width of meristem
d = 0.001  # Diffusion coefficient (of metabolites in xylem fluid)
Rf = 0.04 # Critical values of R and G functions
Rg1 = 0.01  # " "
Rg2 = 1  # " "
f0 = 0.0003075  # " "

# Initial Conditions
Rplot = [1.]# Initial Conditions

# Rplot = [Rf / 2.0] # Growth factor starts below critical value
Lplot = [0.1]  # Plant length is 0 at t = 0
Cplot = [1.]
tplot = [0]

# Initiate a grid (for concentration of metabolites)
h = Lplot[0] / (N - 1)  # Spacing of grid
x = np.arange(0, N) * h  # Initial coordinates of grid points
C = np.ones(N)  # Initiate C (concentration of metabolites)
C[0] = 1  # Boundary Condition

# Construct State Vector
state = np.array([Rplot[0], Lplot[0]])


# %% Numerical Method Functions


def rk4(x, C, t, ht, f):
    F1 = f(x, C, t)
    F2 = f(x + (0.5 * ht) * F1, C, t + 0.5 * ht)
    F3 = f(x + (0.5 * ht) * F2, C, t + 0.5 * ht)
    F4 = f(x + ht * F3, C, t + ht)
    xout = x + ht / 6. * (F1 + F4 + 2. * (F2 + F3))
    return xout

b = (ht * d) / (2 * (h ** 2))  # Final parameter (depends on h so must be defined here)

def crank(s, C_in):
    N = len(C_in)
    a = (ht * f(s[0])) / (4 * h)
    M1 = diags([-a - b, 1 + 2 * b, a - b], [-1, 0, 1], shape=(N, N)).toarray()
    M2 = diags([b + a, 1 - 2 * b, b - a], [-1, 0, 1], shape=(N, N)).toarray()
    M = np.linalg.inv(M1).dot(M2)
    C_out = C_in.dot(M)  # written this way because C is a row vector
    return C_out


def derivated(s, C, t):
    aux = s + ((C[-1]) * g(s) * ht - sigma * s * ht) / (0.0001*w)
    return aux


time = 0.
for istep in range(0, tf):  ###  MAIN LOOP  ###
    L0 = np.copy(state[1])
    state[0] = derivated(state[0], C, istep)
    time = (time + ht)
    
    C = crank(state, C)
    dL = state[1] - L0  # Change in L
    dN = int(np.round(dL / h))  # Number of new grid points

    # Apply BCs to fill in new grid points.
    C[0] = 1
    
    Ci = (1 - (g(state[0]) * (h / d))) * C[-1]  # d dC/dx = - g(R)C
    if dL > 0:
        C = np.append(C, dN * [Ci])
    else:
        C[-1] = Ci
    if (istep % 1000) == 0:
        print('Number of iterations completed:', istep)
    Rplot.append(state[0])
    state[1] = Lplot[-1] + f(state[0])
    Lplot.append(state[1])
    Cplot.append(C[-1])
    tplot.append(istep+1)
    
# Non-animated plots of L(t) and R(t)
plt.figure(1)
plt.plot(tplot, Lplot, label='Linear', color='#1f77b4')
#plt.title('Length of Plant L vs. EDD');
plt.xlabel('EDD');
plt.ylabel('Length');
plt.legend(loc='lower right')
plt.grid(True, which='both', axis='both', linestyle='--')
z=max(tplot)
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0.1, 0.74+0.064, 0.064))

plt.figure(2)
plt.plot(tplot, Rplot, color='#1f77b4')
#plt.title('Growth Factor Concentration R vs. EDD');
plt.xlabel('EDD');
plt.ylabel('R')
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))




plt.figure(3)
plt.plot(tplot, Cplot, color='#1f77b4')
#plt.title('Metabolite Concentration C at end point vs. EDD');
plt.xlabel('EDD');
plt.ylabel('C')
plt.grid()
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))

R_1=Rplot
C_1=Cplot



















import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.sparse import diags
from matplotlib.animation import writers
%matplotlib
plt.ion()
p=0.41
q=3.3
k=0.13489
r=0.01
k_=(k/p)**(1/q)
r_=r/q
def f(R,L):
    if 0 <= R <= Rf:
        f = 0
    elif R > Rf:
        f = r_*L*( 1 - ( (L**q) / (k_**q) ) )
    else:
        print('Error: R must be greater than 0')
    return f


def g(R):
    if 0 <= R <= Rg1:
        g = (g0/Rg1) * R
    elif Rg1 < R < Rg2:
        g = g0
    elif R >= Rg2:
        g = 0
    else:
        print('Error: R must be greater than 0')
    return g

g0=0.01
ht = 0.0001
tf = 3000
N = 101

sigma = 0.009  # Growth factor decay coefficient
w = 0.05  # Width of meristem
d = 0.001  # Diffusion coefficient (of metabolites in xylem fluid)
Rf = 0.04 # Critical values of R and G functions
Rg1 = 0.01  # " "
Rg2 = 1  # " "
f0 = 0.0003075  # " "

# Initial Conditions
Rplot = [1.]# Initial Conditions

# Rplot = [Rf / 2.0] # Growth factor starts below critical value
Lplot = [0.1]  # Plant length is 0 at t = 0
Cplot = [1.]
tplot = [0]

# Initiate a grid (for concentration of metabolites)
h = Lplot[0] / (N - 1)  # Spacing of grid
x = np.arange(0, N) * h  # Initial coordinates of grid points
C = np.ones(N)  # Initiate C (concentration of metabolites)
C[0] = 1  # Boundary Condition

# Construct State Vector
state = np.array([Rplot[0], Lplot[0]])


# %% Numerical Method Functions


def rk4(x, C, t, ht, f):
    F1 = f(x, C, t)
    F2 = f(x + (0.5 * ht) * F1, C, t + 0.5 * ht)
    F3 = f(x + (0.5 * ht) * F2, C, t + 0.5 * ht)
    F4 = f(x + ht * F3, C, t + ht)
    xout = x + ht / 6. * (F1 + F4 + 2. * (F2 + F3))
    return xout

b = (ht * d) / (2 * (h ** 2))  # Final parameter (depends on h so must be defined here)

def crank(s, C_in):
    N = len(C_in)
    a = (ht * f(s[0], s[1])) / (4 * h)
    M1 = diags([-a - b, 1 + 2 * b, a - b], [-1, 0, 1], shape=(N, N)).toarray()
    M2 = diags([b + a, 1 - 2 * b, b - a], [-1, 0, 1], shape=(N, N)).toarray()
    M = np.linalg.inv(M1).dot(M2)
    C_out = C_in.dot(M)  # written this way because C is a row vector
    return C_out


def derivated(s, C, t):
    aux = s + ((C[-1]) * g(s) * ht - sigma * s * ht) / (0.0001*w)
    return aux


time = 0.
for istep in range(0, tf):  ###  MAIN LOOP  ###
    L0 = np.copy(state[1])
    state[0] = derivated(state[0], C, istep)
    time = (time + ht)
    
    C = crank(state, C)
    dL = state[1] - L0  # Change in L
    dN = int(np.round(dL / h))  # Number of new grid points

    # Apply BCs to fill in new grid points.
    C[0] = 1
    
    Ci = (1 - (g(state[0]) * (h / d))) * C[-1]  # d dC/dx = - g(R)C
    if dL > 0:
        C = np.append(C, dN * [Ci])
    else:
        C[-1] = Ci
    if (istep % 1000) == 0:
        print('Number of iterations completed:', istep)
    Rplot.append(state[0])
    #state[1] = Lplot[-1] * (1 + (r_*(1-((Lplot[-1]**q)/(k_**q))) ))
    state[1] = Lplot[-1] + f(state[0], Lplot[-1])
    Lplot.append(state[1])
    Cplot.append(C[-1])
    tplot.append(istep+1)
    
# Non-animated plots of L(t) and R(t)

plt.figure(1)
plt.plot(tplot, Lplot, label='Logistic', color='#ff7f0e')
#plt.title('Length of Plant L vs. EDD');
plt.xlabel('EDD');
plt.ylabel('Length');
plt.legend(loc='lower right')
plt.grid()
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0.1, 0.74+0.064, 0.064))

plt.figure(2)
plt.plot(tplot, Rplot, color='#ff7f0e')
#plt.title('Growth Factor Concentration R vs. EDD');
plt.xlabel('EDD');
plt.ylabel('R')
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0.1, 1+0.1, 0.1))



plt.figure(3)
plt.plot(tplot, Cplot, color='#ff7f0e')
#plt.title('Metabolite Concentration C at end point vs. EDD');
plt.xlabel('EDD');
plt.ylabel('C')
plt.grid()
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))


R_2=Rplot
C_2=Cplot



















import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.sparse import diags
from matplotlib.animation import writers
%matplotlib
plt.ion()
p=0.41
q=3.3
k=0.13489
k_=(k/p)**(1/q)
c=0.0055
def f(R,L):
    if 0 <= R <= Rf:
        a = 0
    elif R > Rf:
        a = c*L*(np.log(k_/L))
    else:
        print('Error: R must be greater than 0')
    return a


def g(R):
    if 0 <= R <= Rg1:
        g = (g0/Rg1) * R
    elif Rg1 < R < Rg2:
        g = g0
    elif R >= Rg2:
        g = 0
    else:
        print('Error: R must be greater than 0')
    return g

g0=0.01
ht = 0.0001
tf = 3000
N = 101

sigma = 0.009  # Growth factor decay coefficient
w = 0.05  # Width of meristem
d = 0.001  # Diffusion coefficient (of metabolites in xylem fluid)
Rf = 0.04 # Critical values of R and G functions
Rg1 = 0.01  # " "
Rg2 = 1  # " "
f0 = 0.0031  # " "

# Initial Conditions
Rplot = [1.]# Initial Conditions

# Rplot = [Rf / 2.0] # Growth factor starts below critical value
Lplot = [0.1]  # Plant length is 0 at t = 0
Cplot = [1.]
tplot = [0]

# Initiate a grid (for concentration of metabolites)
h = Lplot[0] / (N - 1)  # Spacing of grid
x = np.arange(0, N) * h  # Initial coordinates of grid points
C = np.ones(N)  # Initiate C (concentration of metabolites)
C[0] = 1  # Boundary Condition

# Construct State Vector
state = np.array([Rplot[0], Lplot[0]])


# %% Numerical Method Functions


def rk4(x, C, t, ht, f):
    F1 = f(x, C, t)
    F2 = f(x + (0.5 * ht) * F1, C, t + 0.5 * ht)
    F3 = f(x + (0.5 * ht) * F2, C, t + 0.5 * ht)
    F4 = f(x + ht * F3, C, t + ht)
    xout = x + ht / 6. * (F1 + F4 + 2. * (F2 + F3))
    return xout

b = (ht * d) / (2 * (h ** 2))  # Final parameter (depends on h so must be defined here)

def crank(s, C_in):
    N = len(C_in)
    a = (ht * f(s[0], s[1])) / (4 * h)
    M1 = diags([-a - b, 1 + 2 * b, a - b], [-1, 0, 1], shape=(N, N)).toarray()
    M2 = diags([b + a, 1 - 2 * b, b - a], [-1, 0, 1], shape=(N, N)).toarray()
    M = np.linalg.inv(M1).dot(M2)
    C_out = C_in.dot(M)  # written this way because C is a row vector
    return C_out


def derivated(s, C, t):
    aux = s + ((C[-1]) * g(s) * ht - sigma * s * ht) / (0.0001*w)
    return aux


time = 0.
for istep in range(0, tf):  ###  MAIN LOOP  ###
    L0 = np.copy(state[1])
    state[0] = derivated(state[0], C, istep)
    time = (time + ht)
    
    C = crank(state, C)
    dL = state[1] - L0  # Change in L
    dN = int(np.round(dL / h))  # Number of new grid points

    # Apply BCs to fill in new grid points.
    C[0] = 1
    
    Ci = (1 - (g(state[0]) * (h / d))) * C[-1]  # d dC/dx = - g(R)C
    if dL > 0:
        C = np.append(C, dN * [Ci])
    else:
        C[-1] = Ci
    if (istep % 1000) == 0:
        print('Number of iterations completed:', istep)
    Rplot.append(state[0])
    #state[1] = Lplot[-1] * (1 + (r_*(1-((Lplot[-1]**q)/(k_**q))) ))
    state[1] = Lplot[-1] + f(state[0], Lplot[-1])
    Lplot.append(state[1])
    Cplot.append(C[-1])
    tplot.append(istep+1)
    
# Non-animated plots of L(t) and R(t)
plt.figure(1)
plt.plot(tplot, Lplot, label='Gompertz', color='#2ca02c')
#plt.title('Length of Plant L vs. EDD');
plt.xlabel('EDD');
plt.ylabel('Length');
plt.legend(loc='lower right')
plt.grid()
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0.1, 0.74+0.064, 0.064))

plt.figure(2)
plt.plot(tplot, Rplot, color='#2ca02c')
#plt.title('Growth Factor Concentration R vs. EDD');
plt.xlabel('EDD');
plt.ylabel('R')
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))






plt.figure(3)
plt.plot(tplot, Cplot, color='#2ca02c')
#plt.title('Metabolite Concentration C at end point vs. EDD');
plt.xlabel('EDD');
plt.ylabel('C')
plt.grid()
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))



R_3=Rplot
C_3=Cplot













import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.sparse import diags
from matplotlib.animation import writers
%matplotlib
plt.ion()
n=800
p=0.41
q=3.3
KF=0.59
r=0.033
r_=r/q
def f(R,L):
    if 0 <= R <= Rf:
        f = 0
    elif R > Rf:
        f = (r_*L*( (1 - np.exp(-n*KF*p*(L**q) )))) / (n*KF*p*(L**q)) 
    else:
        print('Error: R must be greater than 0')
    return f


def g(R):
    if 0 <= R <= Rg1:
        g = (g0/Rg1) * R
    elif Rg1 < R < Rg2:
        g = g0
    elif R >= Rg2:
        g = 0
    else:
        print('Error: R must be greater than 0')
    return g

g0=0.01
ht = 0.0001
tf = 3000
N = 101

sigma = 0.009  # Growth factor decay coefficient
w = 0.05  # Width of meristem
d = 0.001  # Diffusion coefficient (of metabolites in xylem fluid)
Rf = 0.04 # Critical values of R and G functions
Rg1 = 0.01  # " "
Rg2 = 1  # " "

# Initial Conditions
Rplot = [1.]# Initial Conditions

# Rplot = [Rf / 2.0] # Growth factor starts below critical value
Lplot = [0.1]  # Plant length is 0 at t = 0
Cplot = [1.]
tplot = [0]

# Initiate a grid (for concentration of metabolites)
h = Lplot[0] / (N - 1)  # Spacing of grid
x = np.arange(0, N) * h  # Initial coordinates of grid points
C = np.ones(N)  # Initiate C (concentration of metabolites)
C[0] = 1  # Boundary Condition

# Construct State Vector
state = np.array([Rplot[0], Lplot[0]])


# %% Numerical Method Functions


def rk4(x, C, t, ht, f):
    F1 = f(x, C, t)
    F2 = f(x + (0.5 * ht) * F1, C, t + 0.5 * ht)
    F3 = f(x + (0.5 * ht) * F2, C, t + 0.5 * ht)
    F4 = f(x + ht * F3, C, t + ht)
    xout = x + ht / 6. * (F1 + F4 + 2. * (F2 + F3))
    return xout

b = (ht * d) / (2 * (h ** 2))  # Final parameter (depends on h so must be defined here)

def crank(s, C_in):
    N = len(C_in)
    a = (ht * f(s[0], s[1])) / (4 * h)
    M1 = diags([-a - b, 1 + 2 * b, a - b], [-1, 0, 1], shape=(N, N)).toarray()
    M2 = diags([b + a, 1 - 2 * b, b - a], [-1, 0, 1], shape=(N, N)).toarray()
    M = np.linalg.inv(M1).dot(M2)
    C_out = C_in.dot(M)  # written this way because C is a row vector
    return C_out


def derivated(s, C, t):
    aux = s + ((C[-1]) * g(s) * ht - sigma * s * ht) / (0.0001*w)
    return aux


time = 0.
for istep in range(0, tf):  ###  MAIN LOOP  ###
    L0 = np.copy(state[1])
    state[0] = derivated(state[0], C, istep)
    time = (time + ht)
    
    C = crank(state, C)
    dL = state[1] - L0  # Change in L
    dN = int(np.round(dL / h))  # Number of new grid points

    # Apply BCs to fill in new grid points.
    C[0] = 1
    
    Ci = (1 - (g(state[0]) * (h / d))) * C[-1]  # d dC/dx = - g(R)C
    if dL > 0:
        C = np.append(C, dN * [Ci])
    else:
        C[-1] = Ci
    if (istep % 1000) == 0:
        print('Number of iterations completed:', istep)
    Rplot.append(state[0])
    #state[1] = Lplot[-1] * (1 + (r_*(1-((Lplot[-1]**q)/(k_**q))) ))
    state[1] = Lplot[-1] + f(state[0], Lplot[-1])
    Lplot.append(state[1])
    Cplot.append(C[-1])
    tplot.append(istep+1)
    
# Non-animated plots of L(t) and R(t)
plt.figure(1)
plt.subplots_adjust(left=0.145, bottom=0.108, right=0.988, top=0.94, wspace=0.246)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.plot(tplot, Lplot, label='Scaife & Jones', color='#d62728')
#plt.title('Length of Plant L vs. EDD');
plt.xlabel('EDD');
plt.ylabel('Length');
plt.legend(loc='lower right')
plt.grid()
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0.1, 0.74+0.064, 0.064))

plt.figure(2)
plt.subplots_adjust(left=0.145, bottom=0.108, right=0.988, top=0.94, wspace=0.246)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.plot(tplot, Rplot, color='#d62728')
#plt.title('Growth Factor Concentration R vs. EDD');
plt.xlabel('EDD');
plt.ylabel('R')
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))






plt.figure(3)
plt.subplots_adjust(left=0.145, bottom=0.108, right=0.988, top=0.94, wspace=0.246)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.plot(tplot, Cplot, color='#d62728')
#plt.title('Metabolite Concentration C at end point vs. EDD');
plt.xlabel('EDD');
plt.ylabel('C')
plt.grid()
z=max(tplot)
plt.grid(True, which='both', axis='both', linestyle='--')
plt.xticks(np.arange(0, z+z/3, z/3))
plt.yticks(np.arange(0, 1+0.1, 0.1))




R_4=Rplot
C_4=Cplot
plt.show()






Using matplotlib backend: <object object at 0x000001F49562F300>
Number of iterations completed: 0
Number of iterations completed: 1000
Number of iterations completed: 2000
Using matplotlib backend: TkAgg
Number of iterations completed: 0
Number of iterations completed: 1000
Number of iterations completed: 2000
Using matplotlib backend: TkAgg
Number of iterations completed: 0
Number of iterations completed: 1000
Number of iterations completed: 2000
Using matplotlib backend: TkAgg
Number of iterations completed: 0
Number of iterations completed: 1000
Number of iterations completed: 2000
